In [2]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import datetime as datetime
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

print("Python Version:", sys.version)

Python Version: 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]


In [3]:
df= pd.read_csv('cleaned_imdb_mojo.csv')

In [4]:
df.head()

,Unnamed: 0,Rank_MOJO_by_Box_office,Title,Lifetime_Gross_MOJO,Max_Theaters,Opening_Box,Opening_Th,Distributor,User_Rating_IMDB,Metacritic,Number_of_votes_IMDB,Running_Time,Director,Release_Difference
0,0,1,Fahrenheit 9/11,119194771,2011.0,23920637.0,868.0,Lionsgate,7.5,67.0,125629,2h 2min,Michael Moore,8475.0
1,1,2,March of the Penguins,77437223,2506.0,137492.0,4.0,Warner Independent Pictures (WIP),7.5,79.0,55375,1h 20min,Luc Jacquet,8841.0
2,2,3,Justin Bieber: Never Say Never,73013910,3118.0,29514054.0,3105.0,Paramount Pictures,1.6,NaN,75071,1h 45min,Jon M. Chu,10899.0
3,3,4,This Is It,72091016,3481.0,23234394.0,3481.0,Sony Pictures Entertainment (SPE),7.2,67.0,38161,1h 51min,Kenny Ortega,10428.0
4,4,5,Under the Sea 3D,36262926,108.0,661185.0,49.0,Warner Bros.,7.5,77.0,1730,41min,Howard Hall,10171.0


In [5]:
"""
This was done to analyze the different datasets.
The imdb_mojo dataset was prepared and chosen for further analysis
"""
#Read the four available datasets and rename columns. 
data_bxoff = pd.read_pickle('box_office.pkl')
data_topdoc = pd.read_pickle('top_doc.pkl')
data_imdb = pd.read_csv('IMDB_byVote.csv')
data_mojo = pd.read_pickle('mojo.pkl')
data_bxoff.rename(columns={'Rank':'Rank_BX', 'Year':'Year', 
                          'Title':'Title', 'Worldwide Box Office':'Worldwide_Box_Office_BX',
                          'Domestic Box Office':'Domestic_Box_Office_BX', 
                          'International Box Office':'International_Box_Office_BX'}, inplace = True)
data_topdoc.rename(columns = {'Title':'Title', 'User_Rating_Top_Doc':'User Rating_TD', 'Years':'Year_TD',
                              'Category':'Category'}, inplace = True)
data_imdb.rename(columns ={'Title':'Title', 'User Rating_IMDB':'User_Rating_IMDB', 'Metacritic':'Metacritic',
                               'Number of Votes_IMDB':'Number_of_votes_IMDB', 'Running Time':'Running_Time', 
                               'Release Date_IMDB':'Release_Date_IMDB', 'Director':'Director'}, inplace = True)
data_mojo.rename(columns = {'Mojo Rank':'Rank_MOJO_by_Box_office', 'Title':'Title', 'Lifetime Gross':'Lifetime_Gross_MOJO',
                            'Max Theaters':'Max_Theaters', 'Opening':'Opening_Box', 'Opening Th':'Opening_Th', 'Release Date':'Release_Date',
                            'Distributor':'Distributor'}, inplace = True)

#Drop uncessary columns
data_imdb.drop('Unnamed: 0', axis = 1, inplace = True)
#Strip uncessary characters in distributor column
data_mojo = data_mojo.apply(lambda x: x.str.strip('\n \n') if x.dtype == "object" else x)

#Examine the shape of the datasets
print('The shape of the IMDB data is: {}.'.format(data_imdb.shape))
print('The shape of Box Office Mojo data is: {}'.format(data_mojo.shape))
print('The shape of Top Documentary data is: {}.'.format(data_topdoc.shape))
print('The shape of the-numbers box office data is: {}'.format(data_bxoff.shape))

#Datasets were merged with IMDB dataset on title
#Merging on titles caused the dataset to become smaller
#IMDB merged with Box Office Mojo data gave the largest dataset to work with, 715 rows.
imdb_topdoc = pd.merge(data_topdoc, data_imdb, on=['Title'])
imdb_bxoff = pd.merge(data_bxoff, data_imdb, on=['Title'])
imdb_mojo = pd.merge(data_mojo, data_imdb, on=['Title'])

print('The shape of IMDB - Box Office Mojo data is: {}'.format(imdb_mojo.shape))
print('The shape of IMDB - Top Documentary data is: {}.'.format(imdb_topdoc.shape))
print('The shape of IMDB - the-numbers box office data is: {}'.format(imdb_bxoff.shape))

The shape of the IMDB data is: (2000, 7).
The shape of Box Office Mojo data is: (2075, 8)
The shape of Top Documentary data is: (2549, 4).
The shape of the-numbers box office data is: (1211, 6)
The shape of IMDB - Box Office Mojo data is: (715, 14)
The shape of IMDB - Top Documentary data is: (172, 10).
The shape of IMDB - the-numbers box office data is: (422, 12)


In [6]:
imdb_mojo_topdoc = pd.merge(data_topdoc, imdb_mojo, on=['Title'])

In [8]:
imdb_mojo_topdoc.head()


,Title,User Rating - Top Doc,Year_TD,Category,Rank_MOJO_by_Box_office,Lifetime_Gross_MOJO,Max_Theaters,Opening_Box,Opening_Th,Release_Date,Distributor,User_Rating_IMDB,Metacritic,Number_of_votes_IMDB,Running_Time,Release_Date_IMDB,Director
0,The Hacker Wars,8.10,2014,Technology,"1,937","$3,470",1,"$2,958",1,"Oct 17, 2014",Vitagraph Films,6.5,NaN,931,1h 31min,17 October 2014 (USA),Vivien Lesnik Weisman
1,Zero Days,8.51,2015,Technology,776,"$109,649",20,"$49,110",18,"Jul 8, 2016",Magnolia Pictures,7.8,77.0,"8,679",1h 56min,8 July 2016 (USA),Alex Gibney
2,Revolution OS,8.53,2001,Technology,"1,934","$3,500",1,"$3,500",1,"Aug 23, 2002",Seventh Art Releasing,7.3,NaN,"2,372",1h 25min,15 February 2002 (USA),J.T.S. Moore
3,More Than a Game,6.77,2011,Sports,219,"$950,675",111,"$182,943",14,"Oct 2, 2009",Lionsgate,7.6,NaN,"5,073",1h 45min,16 October 2009 (Taiwan),Kristopher Belman
4,Deep Water,7.94,2006,Sports,474,"$271,143",17,"$22,632",2,"Aug 24, 2007",IFC Films,7.8,80.0,"2,932",1h 32min,15 December 2006 (UK),Louise Osmond


In [11]:
imdb_mojo.head()

,Rank_MOJO_by_Box_office,Title,Lifetime_Gross_MOJO,Max_Theaters,Opening_Box,Opening_Th,Release_Date,Distributor,User_Rating_IMDB,Metacritic,Number_of_votes_IMDB,Running_Time,Release_Date_IMDB,Director
0,1,Fahrenheit 9/11,"$119,194,771","2,011","$23,920,637",868,"Jun 23, 2004",Lionsgate,7.5,67.0,"125,629",2h 2min,25 June 2004 (USA),Michael Moore
1,2,March of the Penguins,"$77,437,223","2,506","$137,492",4,"Jun 24, 2005",Warner Independent Pictures (WIP),7.5,79.0,"55,375",1h 20min,22 July 2005 (USA),Luc Jacquet
2,3,Justin Bieber: Never Say Never,"$73,013,910","3,118","$29,514,054","3,105","Feb 11, 2011",Paramount Pictures,1.6,NaN,"75,071",1h 45min,11 February 2011 (USA),Jon M. Chu
3,4,This Is It,"$72,091,016","3,481","$23,234,394","3,481","Oct 28, 2009",Sony Pictures Entertainment (SPE),7.2,67.0,"38,161",1h 51min,28 October 2009 (USA),Kenny Ortega
4,5,Under the Sea 3D,"$36,262,926",108,"$661,185",49,"Feb 13, 2009",Warner Bros.,7.5,77.0,"1,730",41min,13 February 2009 (USA),Howard Hall


In [12]:
#Save IMDB_mojo merged dataset as CSV 
#imdb_mojo.to_csv('imdb_mojo.csv')
# imdb_mojo= pd.read_csv('imdb_mojo.csv')

In [13]:
print('The shape of IMDB merged with Box Office Mojo data is: {}'.format(df.shape))

The shape of IMDB merged with Box Office Mojo data is: (715, 14)


In [14]:
#Clean data by removing '$',',', and '-'. 
#Convert values from string to numeric
imdb_mojo['Lifetime_Gross_MOJO'] = imdb_mojo['Lifetime_Gross_MOJO'].replace({'\$':''}, regex = True)
imdb_mojo['Lifetime_Gross_MOJO'] = imdb_mojo['Lifetime_Gross_MOJO'].replace({',':''}, regex = True)
imdb_mojo['Lifetime_Gross_MOJO'] = imdb_mojo['Lifetime_Gross_MOJO'].replace({'-':''}, regex = True)
imdb_mojo['Lifetime_Gross_MOJO'] = pd.to_numeric(imdb_mojo['Lifetime_Gross_MOJO'])

imdb_mojo['Opening_Box'] = imdb_mojo['Opening_Box'].replace({'\$':''}, regex = True)
imdb_mojo['Opening_Box'] = imdb_mojo['Opening_Box'].replace({',':''}, regex = True)
imdb_mojo['Opening_Box'] = imdb_mojo['Opening_Box'].replace({'-':''}, regex = True)
imdb_mojo['Opening_Box'] = pd.to_numeric(imdb_mojo['Opening_Box'])

imdb_mojo['Number_of_votes_IMDB'] = imdb_mojo['Number_of_votes_IMDB'].replace({',':''}, regex = True)
imdb_mojo['Number_of_votes_IMDB'] = pd.to_numeric(imdb_mojo['Number_of_votes_IMDB'])

imdb_mojo['Rank_MOJO_by_Box_office'] = imdb_mojo['Rank_MOJO_by_Box_office'].replace({',':''}, regex = True)
imdb_mojo['Rank_MOJO_by_Box_office'] = pd.to_numeric(imdb_mojo['Rank_MOJO_by_Box_office'])

imdb_mojo['Max_Theaters'] = imdb_mojo['Max_Theaters'].replace({',':''}, regex = True)
imdb_mojo['Max_Theaters'] = imdb_mojo['Max_Theaters'].replace({'-':''}, regex = True)
imdb_mojo['Max_Theaters'] = pd.to_numeric(imdb_mojo['Max_Theaters'])

imdb_mojo['Opening_Th'] = imdb_mojo['Opening_Th'].replace({'-':''}, regex = True)
imdb_mojo['Opening_Th'] = imdb_mojo['Opening_Th'].replace({',':''}, regex = True)
imdb_mojo['Opening_Th'] = pd.to_numeric(imdb_mojo['Opening_Th'])

imdb_mojo['User_Rating_IMDB'] = pd.to_numeric(imdb_mojo['User_Rating_IMDB'])
imdb_mojo['Metacritic'] = pd.to_numeric(imdb_mojo['Metacritic'])



In [15]:
#Both IMDB and Box Office Mojo datasets had release date, drop one of them
imdb_mojo.drop(['Release_Date_IMDB',], inplace = True, axis = 1)

In [16]:
#Convert to datetime
#Use time delta to find difference between the select documentary and the oldest documentary in the list
imdb_mojo['Release_Date'] = pd.to_datetime(imdb_mojo['Release_Date']) 
imdb_mojo['Release_Difference'] = (imdb_mojo['Release_Date']-imdb_mojo['Release_Date'].min())/np.timedelta64(1,'D')
imdb_mojo.drop(['Release_Date'], inplace = True, axis = 1)

In [17]:
imdb_mojo.head()

,Rank_MOJO_by_Box_office,Title,Lifetime_Gross_MOJO,Max_Theaters,Opening_Box,Opening_Th,Distributor,User_Rating_IMDB,Metacritic,Number_of_votes_IMDB,Running_Time,Director,Release_Difference
0,1,Fahrenheit 9/11,119194771,2011.0,23920637.0,868.0,Lionsgate,7.5,67.0,125629,2h 2min,Michael Moore,8475.0
1,2,March of the Penguins,77437223,2506.0,137492.0,4.0,Warner Independent Pictures (WIP),7.5,79.0,55375,1h 20min,Luc Jacquet,8841.0
2,3,Justin Bieber: Never Say Never,73013910,3118.0,29514054.0,3105.0,Paramount Pictures,1.6,NaN,75071,1h 45min,Jon M. Chu,10899.0
3,4,This Is It,72091016,3481.0,23234394.0,3481.0,Sony Pictures Entertainment (SPE),7.2,67.0,38161,1h 51min,Kenny Ortega,10428.0
4,5,Under the Sea 3D,36262926,108.0,661185.0,49.0,Warner Bros.,7.5,77.0,1730,41min,Howard Hall,10171.0


In [ ]:
imdb_mojo.to_csv('cleaned_imdb_mojo.csv')

In [18]:
imdb_topdoc

,Title,User Rating - Top Doc,Year_TD,Category,User_Rating_IMDB,Metacritic,Number_of_votes_IMDB,Running_Time,Release_Date_IMDB,Director
0,AlphaGo,8.06,2017,Technology,7.9,NaN,"4,698",1h 30min,29 September 2017 (USA),Greg Kohs
1,The Hacker Wars,8.10,2014,Technology,6.5,NaN,931,1h 31min,17 October 2014 (USA),Vivien Lesnik Weisman
2,Zero Days,8.51,2015,Technology,7.8,77.0,"8,679",1h 56min,8 July 2016 (USA),Alex Gibney
3,Future by Design,8.59,2006,Technology,8.0,NaN,"1,358",1h 26min,10 June 2006 (USA),William Gazecki
4,Revolution OS,8.53,2001,Technology,7.3,NaN,"2,372",1h 25min,15 February 2002 (USA),J.T.S. Moore
...,...,...,...,...,...,...,...,...,...,...
167,Style Wars,8.23,1983,Art and Artists,8.1,NaN,"2,657",1h 9min,TV Movie 1983,Tony Silver
168,Loose Change 9/11: An American Coup,7.72,2009,9/11,7.2,NaN,"3,450",1h 39min,Video 22 September 2009,Dylan Avery
169,9/11: The Falling Man,7.49,2006,9/11,7.3,NaN,"1,386",1h 20min,TV Movie 16 March 2006,Henry Singer
170,102 Minutes That Changed America,5.90,2008,9/11,8.3,NaN,"2,279",1h 42min,TV Movie 11 September 2008,Nicole Rittenmeyer


In [ ]:
#filter out rows without a metacritic score
#has_metacritic_df = imdb_mojo[imdb_mojo.Metacritic.notnull()]

In [ ]:
#Drop Nas in imdb_mojo dataframe
imdb_mojo_drop_na = imdb_mojo.dropna()

In [ ]:
imdb_mojo_drop_na.describe()

In [ ]:
imdb_mojo_drop_na.corr()

In [ ]:
imdb_mojo_drop_na.head()

In [ ]:
no_metacritic_df = imdb_mojo[imdb_mojo.Metacritic.isnull()]

In [ ]:
no_metacritic_df.shape

In [ ]:
#There are 715 films and 149 distributors, with 353 titles made by 15 companies. Make 15 dummmy variables.
#Categorize distributors that make less than 10 movies as 'other'.
distributor_counts = imdb_mojo['Distributor'].value_counts()
distributor_counts.head(50)

In [ ]:
#Will make the correlation with box office numbers positive, before reversing, lots of negative correlations
#imdb_mojo.Rank_MOJO_by_Box_office =  imdb_mojo.Rank_MOJO_by_Box_office.values[::-1]

"""
def new_rank(df):
    for i in imdb_mojo['Rank_MOJO_by_Box_office']:
        new_rank = 2055-i +1
        imdb_mojo['Rank_MOJO_by_Box_office'][i] = new_rank
#imdb_mojo.Rank_MOJO_by_Box_office =  imdb_mojo.Rank_MOJO_by_Box_office.values(2055-the current rank +1)
new_rank(imdb_mojo)
"""


In [ ]:
#Will drop to 524 rows.
imdb_mojo = imdb_mojo.dropna()

In [ ]:
has_metacritic_df.corr()

In [ ]:
'''
data_bxoff.to_csv('data_bxoff.csv')
data_topdoc.to_csv('data_topdoc.csv')
data_imdb.to_csv('data_imdb.csv')
data_mojo.to_csv('data_mojo.csv')

In [ ]:
has_metacritic_df.describe()

In [ ]:
sns.pairplot(imdb_mojo_drop_na, kind = 'scatter')

In [ ]:
sns.pairplot(imdb_mojo_drop_na, kind = 'kde')